In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Fri Apr 30 13:26:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install emoji

     |████████████████████████████████| 133kB 5.8MB/s 


In [5]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
class_weights_tensor = torch.FloatTensor(class_weights)

In [6]:
import torch.nn.functional as f
class DAN(nn.Module):
    def __init__(self, layer_dims, embedding_dim, vocab_size, pad_token):
        super(DAN, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linears = nn.ModuleList([])
        self.pad_token = pad_token

        input_size = embedding_dim
        for hidden_size in layer_dims:
          self.linears.append(nn.Linear(input_size, hidden_size))
          input_size = hidden_size

        self.hidden_dropout = nn.Dropout(0.5)
        self.output_layer = nn.Linear(input_size, 3)

    def forward(self, X, train=False):
        X_mask = X != self.pad_token
        X_len = torch.sum(X_mask, dim=0)
        x = self.embeddings(X)
        x = torch.sum(x, dim=0) / X_len.view(X_len.size()[0], 1)

        for l in self.linears:
          x = l(x)
          x = f.relu(x)
          x = self.hidden_dropout(x)

        return self.output_layer(x)


In [7]:
from sklearn.metrics import f1_score
import random

def score_model(model, test_ds):
  model.eval()
  with torch.no_grad():
    test_dl = DataLoader(dataset=test_ds, batch_size=len(test_ds.samples),
      shuffle=False, collate_fn=StockSentimentDataset.collate_fn)
    predictions = []
    for x, y in test_dl:
      x = x.cuda()
      y = y.cuda()
      pred_scores = model(x)
      predictions.extend(torch.argmax(pred_scores, dim=1).tolist())
  return f1_score(test_ds.labels, predictions, labels=[0,1,2], average='micro')


In [10]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix


def train(model, best_val_model, train_ds, val_ds, class_weights):
  ### Note: note using class_weights in the loss function since that gives worse performance
  class_weights = class_weights.cuda()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  train_dl = DataLoader(dataset=train_ds, batch_size=len(train_ds.samples),
    shuffle=True, collate_fn=StockSentimentDataset.collate_fn)

  val_dl = DataLoader(dataset=val_ds, batch_size=len(val_ds.samples),
    shuffle=False, collate_fn=StockSentimentDataset.collate_fn)

  epoch_stats = []
  best_f1 = 0.0
  n_epochs_without_improvement = 0
  best_val_model.load_state_dict(model.state_dict())
  for epoch in range(250):
    totalLoss = 0.0
    n_samples = 0
    model.train()
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      x = x.cuda()
      y = y.cuda()
      model.zero_grad()
      predicted_scores = model(x)
      loss = loss_fn(predicted_scores, y)
      loss.backward()
      totalLoss += loss.item()
      n_samples += x.size()[1]
      optimizer.step()


    average_train_loss = totalLoss / float(n_samples)

    model.eval()
    total_val_loss = 0.0
    n_val_samples = 0
    with torch.no_grad():
      for x, y in val_dl:
        x = x.cuda()
        y = y.cuda()
        predicted_scores = model(x)
        loss = loss_fn(predicted_scores, y)
        total_val_loss += loss.item()
        n_val_samples += x.size()[1]
    
    average_val_loss = total_val_loss / float(n_val_samples)
    epoch_stats.append((epoch, average_train_loss, average_val_loss))
    val_f1 = score_model(model, val_ds)
    if epoch % 10 == 0:
      print("Epoch {0} finished. Train loss: {1}, Val loss: {2}".format(epoch, average_train_loss, average_val_loss))

    if val_f1 > best_f1:
      best_f1 = val_f1
      n_epochs_without_improvement = 0
      best_val_model.load_state_dict(model.state_dict())
    # else:
    #   n_epochs_without_improvement += 1
    #   if n_epochs_without_improvement == 10:
    #     break

  return best_f1, epoch_stats

model = DAN([120], 100, featurizer.vocab_size, featurizer.PADDING_ID)
best_val_model = DAN([120], 100, featurizer.vocab_size, featurizer.PADDING_ID)
add_embeddings_to_embedding_matrix(embeddings, model.embeddings, featurizer)
model = model.cuda()
best_val_model = best_val_model.cuda()
train(model, best_val_model, train_ds, val_ds, class_weights_tensor)

Epoch 0 finished. Train loss: 0.0012811448576689028, Val loss: 0.005575903216186835


Epoch 10 finished. Train loss: 0.0011794405801854905, Val loss: 0.005445291801374786


Epoch 20 finished. Train loss: 0.001164165922779717, Val loss: 0.005411072044956441


Epoch 30 finished. Train loss: 0.0011464617039897654, Val loss: 0.005394025116550679


Epoch 40 finished. Train loss: 0.0011267751225806175, Val loss: 0.005371671550127925


...

Epoch 210 finished. Train loss: 0.0001325374110565983, Val loss: 0.005920887601618864


Epoch 220 finished. Train loss: 0.00011059123821909016, Val loss: 0.006106060378405513


Epoch 230 finished. Train loss: 9.033297843272143e-05, Val loss: 0.0063467725199096055


Epoch 240 finished. Train loss: 7.653638015616975e-05, Val loss: 0.0064411461353302


(0.5510204081632653,
 [(0, 0.0012811448576689028, 0.005575903216186835),
  (1, 0.001261643975219055, 0.005546931101351368),
  (2, 0.0012393088230598878, 0.005515236027386724),
  (3, 0.0012334023896354784, 0.005505651235580444),
  (4, 0.0012135421744536514, 0.005504982203853374),
  (5, 0.0011897312663700452, 0.005496476985970322),
  (6, 0.0011982448292512968, 0.005483074455845113),
  (7, 0.001183334881454149, 0.005470277703538233),
  (8, 0.0011822475720815796, 0.00546115332720231),
  (9, 0.0011963366937584872, 0.005452067268138029),
  (10, 0.0011794405801854905, 0.005445291801374786),
  (11, 0.0011895623537573484, 0.005445753433266465),
  (12, 0.0011686646636694309, 0.005443630169849007),
  (13, 0.0011724002815053538, 0.005438276699611119),
  (14, 0.0011713273978993968, 0.005429202804760057),
  (15, 0.00116657909124729, 0.005422639603517494),
  (16, 0.0011650241247498163, 0.005415950502668109),
  (17, 0.001162619349455545, 0.005411913200300567),
  (18, 0.0011595262028072009, 0.005410018

In [11]:
print(score_model(model, train_ds))
print(score_model(model, test_ds))
print(score_model(model, val_ds))

print(score_model(best_val_model, train_ds))
print(score_model(best_val_model, test_ds))
print(score_model(best_val_model, val_ds))


0.9988998899889989
0.5794871794871795
0.5306122448979592
0.9647964796479648
0.5897435897435898
0.5510204081632653
